In [1]:
import itertools
import operator
import gensim
import konlpy
import pickle
import pandas as pd
import scipy as sp 
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances

# Crowd Funding Recommender system

## Idea
- For new crowd funding projects, there are no such classificable factors with old projects (I suppose category is not a key factor)
- Users are only interested in a project itself (Funding target money, duration, etc are non of their business)
- Doc2vec is a quite logical method to classify documents
- Classifying descriptions of projects by using Doc2vec -> Recommending 10 nearest projects with a project users clicked or supported

## Outline
- Tokenizing description
- Run Doc2vec
- Calculate distance of each project combinations
- Recommend

In [2]:
cf_df = pd.read_excel('cf_description.xlsx')

## 1. Tokenization

In [3]:
# Select morphemes
feature_pos = ('NNP', 'NNG', 'NNB', 'NP', 'VV', 'VA', 'MAG')

In [4]:
# wrapper
kkma = Kkma()

In [ ]:
tags = []
kon_words = []
kon_words_tags = []
# tokenization 
for i in cf_df['description'][0]:
    kon_word = kkma.pos(i)
    kon_words.append(kon_word)

# morphemes
des_token =[]
for i in range(0, len(kon_words)):
    tokens = []
    for j in range(0, len(kon_words[i])):
        if kon_words[i][j][1] in feature_pos:
            tokens.append(kon_words[i][j][0])
    des_token.append(tokens)

In [ ]:
# transform requirements of doc2vec (words=[], tags=[])
kon_token_tags=[]
for i in np.arange(len(kon_words)):
    kon_token_tag = gensim.models.doc2vec.LabeledSentence(words = des_token[i], tags = [tb_df['id'][i]])
    kon_token_tags.append(kon_token_tag)

In [ ]:
#token data save
output = open('des_token.pkl', 'wb')
pickle.dump(des_token, output)
output.close()

In [5]:
#token data load
pkl_file = open('des_token.pkl', 'rb')
des_token = pickle.load(pkl_file)
pkl_file.close() 

In [ ]:
# token_tags data save
output = open('kon_token_tags.pkl', 'wb')
pickle.dump(kon_token_tags, output)
output.close()

In [6]:
# token_tags data load
pkl_file = open('kon_token_tags.pkl', 'rb')
kon_token_tags = pickle.load(pkl_file)
pkl_file.close() 

## 2. Modeling

In [ ]:
# doc2vec modeling with tokens
cf_doc2vec_model = gensim.models.Doc2Vec(kon_token_tags, size = 200)

In [ ]:
# doc2vec model save (based on konply tokenizer)
cf_doc2vec_model.save('cf_doc2vec_model.model')

In [7]:
# doc2vec model load (based on konply tokenizer)
cf_doc2vec_model = gensim.models.Doc2Vec.load('cf_doc2vec_model.model')

## 3. Caculate distance
- Using euclidean distance

In [8]:
""" euclidean distance """
def dist_cal(doc_model):
    import scipy as sp 
    # make vector bag
    doc_vecs = []
    dists = []
    id_list = []
    eu_dist = sp.spatial.distance.euclidean
    for i in cf_df['id']:
        vec = doc_model.infer_vector('{number}'.format(number=i))
        id_list.append(i)
        doc_vecs.append(vec)
    # caculate distance
    for a, b in itertools.combinations(np.arange(len(doc_vecs)), 2):
        dist = eu_dist(doc_vecs[a], doc_vecs[b])
        dists.append((id_list[a], id_list[b], dist))
    return dists

In [9]:
# distance calculation into dataframe
token_df = pd.DataFrame(dist_cal(cf_doc2vec_model))

## 4. Recommend projects

In [10]:
"""recommend project list"""
def project_recommend(dist_df, number):
    import pandas as pd
    recommend_list = pd.DataFrame()
    if number <= 2000:
        recommend = dist_df.loc[dist_df[0] == number]
        sorting = recommend.sort_values(by=[2], ascending = False)[:11]
        sorting.index = np.arange(len(sorting))
        sorting_df = sorting.drop([0], axis=1)
        for i in sorting_df[1]:
            pj = cf_df.loc[cf_df['id'] == i]
            recommend_list = recommend_list.append(pj)
        recommend_list.index = np.arange(len(recommend_list))
        return recommend_list

    else:
        recommend = dist_df.loc[dist_df[1] == number]
        sorting = recommend.sort_values(by=[2], ascending = False)[:11]
        sorting.index = np.arange(len(sorting))
        sorting_df = sorting.drop([1], axis=1)
        for i in sorting_df[0]:
            pj = cf_df.loc[cf_df['id'] == i]
            recommend_list = recommend_list.append(pj)
        recommend_list.index = np.arange(len(recommend_list))
        return recommend_list

- project_recommend(distance_dataframe, project_id)

In [11]:
cf_df.ix[0]

id                                                                 12
blurb                               세상 단 한 사람을 위한 동화책. <사슴을 타고 온 아이.>
title                                                     사슴을 타고 온 아이
category                                                           동화
end_with_success                                                 True
has_video                                                       False
permalink                                                 /dear_child
description         \n      저를 모르시는 모든 분들께 인사드릴까요. 김가경이라고 합니다.영화와 ...
Name: 0, dtype: object

In [14]:
project_recommend(token_df, 12)

,id,blurb,title,category,end_with_success,has_video,permalink,description
0,2417,전방위.무정형.비연애인구 계간홀로입니다. 잊을 만하면 찾아오는 계간홀로는 당신의 미저리♥,비연애인구 전용잡지 '계간홀로' 6호 발간,출판,True,False,/quarterlyalone6,\n\n전방위. 무정형. 비연애인구 전용잡지 계간홀로가 돌아왔습니다. \n홀로들의 ...
1,1942,개그공연 실루엣 전주편! 많은 분들께 웃음을 전해드리겠습니다!,전주 개그공연 실루엣,연극,False,False,/silhouettecrew,\n안녕하세요 개그팀 실루엣 입니다!\n저희는 5명의 개그맨 지망생(아직은)으로 이...
2,3766,밟혀 지나치는 것들을 코믹하게 풀어나가는 캐릭터 악세사리,짜부 (ZZABOO),제품디자인,False,True,/zzaboo,\n누구세요?\n저희는 단편 애니메이션 제작소 날씨가게 입니다!!\r\n날씨가게의 ...
3,52,"블랙 코미디 성인용 애니메이션. 따뜻한 가족애와 웃음, 풍자가 가득합니다.",Johnny Macarthy 파일럿 프로젝트,애니메이션,False,False,/johnny-macarthy,\n블랙 코미디 애니메이션 Johnny Macarthy를 소개합니다.\n\n시놉시스...
4,4782,"아이의 눈을 읽는, 어른의 마음을 담은 그림이 있는 그림가게 입니다.",그림가게의 이야기가 있는 포스터,미술,False,False,/geurimgage,"\n안녕하세요. 아이들의 눈을 읽는 그림,\n어른의 마음을 담는 그림이 있는 그림가..."
5,130,"편견 없고 편식하지 않는, 복합예술매거진을 지향하는 ""ARTZINE"" 4호의 인쇄비...",아트진[ARTZINE] 4호 출간 프로젝트,저널리즘,True,False,/artzine,"\n여러분!!!!우선, 클릭해주셔서 진심으로 감사 드립니다.^^\n 안녕하세요, 아..."
6,4596,싱어송라이터 '이설아'가 첫번째 EP앨범을 통해 순간을 기록하려 합니다. ...,"나, 그리고 당신. 별이 내리는 길목에서",음악,False,False,/leesnowchild,"\nPhoto by FOUND magazine \n\n나, 그리고 당신. \r..."
7,3420,단편영화 [스프전쟁]의 보충촬영 + 후반작업을 위한 여러분의 즐거움이 필요합니다!,단편영화 [스프전쟁],단편영화,False,False,/soupwar,"\n· 개요 및 인사말\n안녕하십니까, 저희는 단편영화 <스프전쟁>팀으로, 이번 작..."
8,1802,유튜브 동영상 강의를 통한 한국어 수출 프로젝트,이지 코리안 - 동영상 강의로 한국어 수출하기,웹시리즈,False,True,/easykorean,\n두유 노우 한국말?\n한국인을 제외하고 세계인구의 0.079%만 할 줄 아는 한...
9,1667,마왕이 되기 위해 던전을 꾸미고 용사를 물리쳐라!\r\r\n히어로 디텍티드의 확장판...,보드 게임 Hero Detected의 확장판 TIME TWIST!,보드게임,True,False,/hdtt,\n진정한 마왕이 되기 위해 던전을 꾸미고 용사를 물리치는 보드게임\r\n히어로 디...
